In [3]:
# Прянишников Александр, ИУ7-35Б
# Лабораторная работа №4: классификация языков

# Я не успел подсчитать всё, что хотел, так как часто были проблемы с оперативной памятью, и считается всё очень долго
# Поэтому я просто буду обновлять комментарии, прописывая итоги моей работы
# Перед дедлайном у меня был score 0.94, итоговое 4 место в соревновании.
# Дальнейшие мои комментарии и действия - исключительно мой интерес.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

!pip install langdetect

from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from langdetect import detect
import nltk

In [4]:
# Устанавливаем константы, скачиваем данные
SEED = 42
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
# Сначала я пробовал очистить выборку с помощью библиотеки Langdetect
# Считалось 8 часов, но в итоге оказалось, что ошибок стало ещё больше
# Но код я оставлю

for i in range(train_df.shape[0]):
    lang = train_df["language"][i]
    
    # Добавил исключения, так как в гугловской библиотеке только 55 языков
    # И если язык не определён, то кидается исключение
    try:
        google_lang = detect(train_df["sentence"][i])
        if (lang != google_lang):
            train_df["language"][i] = google_lang
    except:
        pass
    
# У такого подхода два минуса
# Во-первых, он некорректно определяет язык, причём главная беда - синтетические ошибки исчезли только на 50%
# Во-вторых, оооочень долго
# Поэтому больше я использовать это не буду

In [5]:
# Тогда я начал вручную смотреть каждый язык, и проверять, если ли в нём синтетические ошибки

# Так я смотрел первые 50 записей, и смотрел, если ли систематические ошибки
train_df[train_df["language"] == "ru"].head(50)

# Так я копировал полную строку, которую потом надо почистать
train_df[train_df["language"] == "ta"]["sentence"][164]

'the and of it that you to apos we this is in they so are'

In [6]:
# Исправим выборку train от искусственных ошибок
sent_1 = "що це тому ми чи дуже які щоб дякую якщо було вона від мене ось"
sent_2 = "гэта што калі мы не яны як але ён каб дзякуй больш пра на вы"
sent_3 = "это что мы не как на вы они но из то он так для аплодисменты"
sent_4 = "笑聲 掌聲 謝謝 所以 現在 事實上 當然 因此 謝謝大家 對吧 但是 鼓掌 謝謝各位 他說 我說"
sent_5 = "笑声 掌声 谢谢 现在 所以 事实上 当然 鼓掌 但是 因此 那么 非常感谢 谢谢大家 是的 好吧"
sent_6 = "що це ми не як на вони та але ви до про він оплески коли"
sent_7 = "бұл мен біз бір үшін ол деп және емес бар керек бірақ олар қол осы"
sent_8 = "the and of it that you to apos we this is in they so are"
for i in range(train_df.shape[0]):
    if (train_df["sentence"][i] == sent_1):
        train_df["language"][i] = "uk"
    elif (train_df["sentence"][i] == sent_2):
        train_df["language"][i] = "uk"
    elif (train_df["sentence"][i] == sent_3):
        train_df["language"][i] = "ru"
    # Насчёт 4 и 5 не уверен, какой из китайских, поставил китайский, который используется в Китае
    # Там очень странное разделение
    elif (train_df["sentence"][i] == sent_4):
        train_df["language"][i] = "zh-cn"
    elif (train_df["sentence"][i] == sent_5):
        train_df["language"][i] = "zh-cn"
    elif (train_df["sentence"][i] == sent_6):
        train_df["language"][i] = "uk"
    elif (train_df["sentence"][i] == sent_7):
        train_df["language"][i] = "kk"
    elif (train_df["sentence"][i] == sent_8):
        train_df["language"][i] = "en"

In [ ]:
# Затем я использовал Tf-IDF и запустил логистическую регрессию
new_train_df = train_df[["sentence", "language"]]
x = new_train_df["sentence"]
y = new_train_df["language"]

LE = LabelEncoder()
y = LE.fit_transform(y)

# Именно под такими параметрами у меня получился наибольший score на Kaggle
# Я пробовал менять ngram, max_df, analyzer
# Получилось, что на символах результат существенно лучше, а max_df не особо влияет на результат
IDF = TfidfVectorizer(analyzer = "char", ngram_range=(2, 3), max_df = 0.4, min_df = 10)
x = IDF.fit_transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=SEED, shuffle = True)

LR = LogisticRegression(class_weight="balanced")
LR.fit(x_train, y_train)
predict = LR.predict(x_test)
balanced_accuracy_score(y_test, predict)

# У меня получилось 0.94 (считал в колабе), на Kaggle практически столько же

In [ ]:
# Сделал submit на Kaggle
x_t = test_df["sentence"]
x_t = IDF.transform(x_t)
new_pred = LR.predict(x_t)

test_df['language'] = LE.classes_[new_pred]
test_df[['index',  'language']].to_csv("second_submission.csv", index=False)

In [ ]:
# Построим матрицу ошибок, чтобы понять, в каких языках ошибается система
fig, ax = plt.subplots(figsize=(100, 100))
plot_confusion_matrix(LR, x_test, y_test, display_labels = LE.classes_, ax = ax)

In [ ]:
# Обнаружены следующие пары ошибок:
# 1: Русский и украинский часто считаются болгарскими
# 2: Программа плохо справляется с балканскими языками, считая словенский и хорватский как боснийский
# 3: Словенский программа считает за чешский
# 4: Нидерладнский часто воспринимается за немецкий
# 5: Некоторые европейские языки часто выдаются за галисийский
# 6: Хорватский - сербский
# 7: Куча ошибок в определении того, какой из языков - какая версия китайского
# 8: fr и fr-cn
# 9: Тайванский часто предсказывается неверно

In [ ]:
# Теперь обучим RF по приколу
RF = RandomForestClassifier(n_estimators = 10, class_weight="balanced")
RF.fit(x_train, y_train)
predict = RF.predict(x_test)
balanced_accuracy_score(y_test, predict)

# Результат получился 0.80, при этом обучение шло очень долго.

In [2]:
# Теперь создадим список стоп-слов
lang = ['hungarian', 'swedish', 'kazakh', 'norwegian', 'finnish', 'arabic', 'indonesian', 'portuguese', 'turkish', 'azerbaijani',
         'slovene', 'spanish', 'danish', 'nepali', 'romanian', 'greek', 'dutch', 'tajik', 'german', 'english', 'russian','french','italian']

stop_words = []
for lan in lang:
    stop_words += nltk.corpus.stopwords.words(lan)

# Впоследствии оказалось, что наличие словаря стоп-слов вообще никак не влияет на качество модели

In [3]:
# Проделаем тоже самое с CountVectorizer
CV = CountVectorizer(analyzer = "char", lowercase=True, ngram_range=(2, 3), stop_words = stop_words, min_df = 10, max_df=0.4)

new_train_df = train_df[["sentence", "language"]]
x = new_train_df["sentence"]
y = new_train_df["language"]

LE = LabelEncoder()
y = LE.fit_transform(y)

x = CV.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=SEED, shuffle = True)

LR = LogisticRegression(class_weight="balanced")
LR.fit(x_train, y_train)
predict = LR.predict(x_test)
balanced_accuracy_score(y_test, predict)

# Для этих действий у меня ни разу не хватило ОЗУ, то есть TF-IDF оказался эффективнее

In [ ]:
# Теперь применим метод ближайших соседей, который будет считать оочень долго, но для языков может оказаться успешным
KNN = KNNNeighborsClassifier()
KNN.fit(x_train, y_train)
predict = KNN.predict(x_test)
balanced_accuracy_score(y_test, predict)

# В итоге у меня так и не подсчиталось даже для одного соседа

In [ ]:
# Также на англоязычных источниках много рассказывали про Наивный Байес
# Попробуем и его

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=SEED, shuffle = True)

MN = MultinomialNB()
MN.fit(x_train, y_train)
pred = MN.predict(x_test)
balanced_accuracy_score(y_test, pred)